In [1]:
#Code to get the word from terminal, obviously not used in this Notebook
#import sys
#word=sys.argv[0]

In [2]:
import math
#from scipy.stats import binom

def read_words(filename):
    f=open(filename)
    counters = {} #initialize empty dictionary
    tot=0
    for line in f: #iterate over the lines of the file
        for word in line.lower().split(): #convert to lower case and split lines
            if word.isalpha(): #check if the split word contains something other than letters
                if word in counters:
                    counters[word]+=1
                else:
                    counters[word]=1
                tot+=1
    f.close()
    priors={}
    for word in counters:
        priors[word]=counters[word]/tot #compute the relative frequency of each word P(C)
    return priors
            
DICT=read_words("big.txt")

def bcoeff(n,k):
    if(n-k>0):
        den=math.factorial(k)*math.factorial(n-k) 
        return math.factorial(n)//den #binomial coefficient
    else:
        return 0 #allow for the number of errors to exceed the lenght of the word without raising errors

def binomial(n,k,p):
    return bcoeff(n,k)*(p**k)*((1.-p)**(n-k)) #return binomial distribution value

def edit1(word): #function to tell the reachable words with 1 error given an input word
    n=len(word)
    letters = "abcdefghijklmnopqrstuvwxyz"
    var=set() #set removes duplicates, so we use it in place of a list
    for i in range(n+1):
        #Insertion
        for l in letters:
            nword=word[:i]+l+word[i:]
            var.add(nword)
            #Substitution of a letter
            if i<n:
                nword=word[:i]+l+word[(i+1):]
                var.add(nword)
        #Deletion
        if i<n:
            nword=word[:i]+word[(i+1):]
            var.add(nword)
        #Consecutive Letters Flip
        if i+1<n:
            nword=word[:i]+word[i+1]+word[i]+word[(i+2):]
            var.add(nword)
    return var

def nedit(word,n): #function to tell the reachable words with 1 error given an input word
    var={word}
    for _ in range(n): #_ is used to indicate an unused variable
        nvar=set()
        for v in var:
            nvar|=edit1(v)
        var=nvar
    return var

word="gor"
#print(len(nedit(word,3)))

def correct_word(word, maxerrors, p, top): #top stand for the number of candidates
    candidates=[]
    for e in range(maxerrors+1):
        var=nedit(word,e)
        for c in var:
            if c in DICT:
                p_c=DICT[c]
                p_wc=binomial(len(c), e, p)
                score = p_c*p_wc
                candidates.append((score,c))
    candidates.sort(reverse=True)
    return candidates[:top]

candidates = correct_word(word,3,0.01,5)
print(candidates)
#print(DICTIONARY)

[(0.00021268793790460801, 'for'), (0.0001132002925732252, 'or'), (1.465529852294189e-05, 'go'), (8.730926518269451e-06, 'nor'), (8.730926518269451e-06, 'got')]
